In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

ziptrain = zipfile.ZipFile("/content/drive/MyDrive/makanantest.zip")
ziptrain.extractall()
ziptrain.close()

In [ ]:
import pandas as pd

csv_file_path = "/content/nutritiondata.csv"
df = pd.read_csv(csv_file_path)

print(df.head())

   Unnamed: 0.1  Unnamed: 0                        food  Caloric Value   Fat  \
0             0           0  chocolate pudding fat free            105   0.3   
1             1           1             tapioca pudding            143   4.3   
2             2           2    tapioca pudding fat free            105   0.4   
3             3           3                rice pudding            122   2.4   
4             4           4                corn pudding            328  12.6   

   Saturated Fats  Monounsaturated Fats  Polyunsaturated Fats  Carbohydrates  \
0             0.0                  0.00                 0.000           23.6   
1             1.1                  2.80                 0.088           23.9   
2             0.1                  0.08                 0.067           23.9   
3             1.4                  0.60                 0.100           20.8   
4             6.3                  3.90                 1.400           42.4   

   Sugars  ...  Calcium   Copper   Iro

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_datasets as tfds
model_baru = load_model("/content/eatwise_model.h5")

In [ ]:
import tensorflow_datasets as tfds

dataset_name = "food101"
dataset_info = tfds.builder(dataset_name).info
class_names = dataset_info.features['label'].names

In [ ]:
import tensorflow_datasets as tfds
import pandas as pd

dataset_name = "food101"
dataset_info = tfds.builder(dataset_name).info
class_names = dataset_info.features['label'].names

for idx, label in enumerate(class_names):
    print(f"{idx}: {label}")

df = pd.DataFrame({
    "Index": range(len(class_names)),
    "Label": class_names
})

output_csv_path = "/content/food101_labels.csv"
df.to_csv(output_csv_path, index=False)

print(f"CSV dengan label tersimpan di: {output_csv_path}")

0: apple_pie
1: baby_back_ribs
2: baklava
3: beef_carpaccio
4: beef_tartare
5: beet_salad
6: beignets
7: bibimbap
8: bread_pudding
9: breakfast_burrito
10: bruschetta
11: caesar_salad
12: cannoli
13: caprese_salad
14: carrot_cake
15: ceviche
16: cheesecake
17: cheese_plate
18: chicken_curry
19: chicken_quesadilla
20: chicken_wings
21: chocolate_cake
22: chocolate_mousse
23: churros
24: clam_chowder
25: club_sandwich
26: crab_cakes
27: creme_brulee
28: croque_madame
29: cup_cakes
30: deviled_eggs
31: donuts
32: dumplings
33: edamame
34: eggs_benedict
35: escargots
36: falafel
37: filet_mignon
38: fish_and_chips
39: foie_gras
40: french_fries
41: french_onion_soup
42: french_toast
43: fried_calamari
44: fried_rice
45: frozen_yogurt
46: garlic_bread
47: gnocchi
48: greek_salad
49: grilled_cheese_sandwich
50: grilled_salmon
51: guacamole
52: gyoza
53: hamburger
54: hot_and_sour_soup
55: hot_dog
56: huevos_rancheros
57: hummus
58: ice_cream
59: lasagna
60: lobster_bisque
61: lobster_roll_sa

In [ ]:
import tensorflow as tf
import numpy as np
import os

def preprocess_image_from_path(image_path):
    img = tf.keras.utils.load_img(image_path)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.image.resize(img_array, (224, 224))
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

In [ ]:
def get_prediction_with_threshold(predictions, class_names, threshold=80):
    max_confidence_idx = np.argmax(predictions)
    max_confidence_score = predictions[max_confidence_idx] * 100

    if max_confidence_score >= threshold:
        return {
            "label": class_names[max_confidence_idx],
            "confidence": max_confidence_score
        }
    else:
        return {
            "label": "Unknown",
            "confidence": 0.0
        }

In [ ]:
def get_nutrition(label, df):
    row = df[df['food'] == label]
    if not row.empty:
        return {
            "calories": row["Caloric Value"].values[0],
            "protein": row["Protein"].values[0],
            "carbs": row["Carbohydrates"].values[0],
            "fats": row["Fat"].values[0],
            "water": row["Water"].values[0],
            "sugars": row["Sugars"].values[0],
            "cholesterol": row["Cholesterol"].values[0],
            "calcium": row["Calcium"].values[0],
            "zinc": row["Zinc"].values[0],
            "magnesium": row["Magnesium"].values[0],
            "sodium": row["Sodium"].values[0]
        }
    else:
        return None

In [ ]:
folder_path = "/content/makanantest/"

In [ ]:
for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, file_name)
        preprocessed_image = preprocess_image_from_path(image_path)

        predictions = model_baru.predict(preprocessed_image)[0]

        result = get_prediction_with_threshold(predictions, class_names, threshold=80)

        if result["label"] != "Unknown":
            nutrition = get_nutrition(result["label"], df)
            if nutrition:
                print(f"Gambar: {file_name} => Label Prediksi: {result['label']} (Confidence: {result['confidence']:.2f}%)")
                print("Informasi Nutrisi:")
                for key, value in nutrition.items():
                    print(f"  {key.capitalize()}: {value}")
            else:
                print(f"Gambar: {file_name} => Label Prediksi: {result['label']} (Confidence: {result['confidence']:.2f}%)")
                print("Informasi nutrisi tidak ditemukan di CSV.")
        else:
            print(f"Gambar: {file_name} => Label Prediksi: Unknown")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Gambar: macaron.jpeg => Label Prediksi: macarons (Confidence: 99.85%)
Informasi Nutrisi:
  Calories: 362
  Protein: 4.8
  Carbs: 49.7
  Fats: 16.4
  Water: 55.0
  Sugars: 0.0
  Cholesterol: 67.3
  Calcium: 15.2
  Zinc: 0.4
  Magnesium: 7.6
  Sodium: 0.3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Gambar: pizza.jpg => Label Prediksi: pizza (Confidence: 99.81%)
Informasi Nutrisi:
  Calories: 136
  Protein: 5.5
  Carbs: 21.8
  Fats: 2.9
  Water: 218.5
  Sugars: 9.5
  Cholesterol: 0.0
  Calcium: 0.0
  Zinc: 0.0
  Magnesium: 2.3
  Sodium: 0.9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Gambar: Spaghetti-Bolognese-daging.jpeg => Label Prediksi: spaghetti_bolognese (Confidence: 99.95%)
Informasi Nutrisi:
  Calories: 255
  Protein: 14.3
  Carbs: 43.1
  Fats: 2.9
  Water: 220.2
  Sugars: 7.4
  Cholesterol: 17.0
  Calcium: 50.9
  Zinc: 1.4
  Magnesium: 42.5
  Sodium: 0.7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Gambar: waffle.jpg => Label Prediksi: waffles (Confidence: 100